In [1]:
import pandas as pd
import numpy as np
import math
import operator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Exploratory Data Analysis

# Training & Test Sets

In [2]:
# import csv
# with open('QueryABCD_130320.csv', newline='') as csvfile:
#      data = list(csv.reader(csvfile))
        
csvfile = pd.read_csv('QueryABCD_130320.csv')
df = pd.DataFrame(csvfile)
# print(df.head())
df.columns = ['Time','ID2','RSSI','dBm','Label']
# print(df.info())
df['Time'] = df['Time'].str.strip()
df['Time'] = pd.to_datetime(df['Time'])
df['Time'].date='20-03-15'
# print("CODE: 1A2B\n", df)

from datetime import timedelta
timeDur = (df['Time'].iloc[-1] - df['Time'].iloc[0]).seconds / 2

df['ID2'] = df['ID2'].astype(str)
df['ID2'] = df['ID2'].apply(int, base=16)
# print(df.info())

# df.isnull().values.any()
# df['Time'].values.tolist()
# groupedView = df.groupby(['Time', 'ID2']) 
# groupedView.first() 


sampleStartTime = df['Time'][0] + timedelta(seconds=2)

# df['Time']
# sampleStartTime
sortdf = df.copy()
sortdf.sort_values(by=['ID2'], inplace=True)

print("CODE: 1T4Z\n", df.head(15))
height = int(timeDur)
resampled = pd.DataFrame(0, index=range(height), columns=sortdf['ID2'].unique())
resampled['Label'] = -1
for columns in resampled:
    resampled[columns] = 0
    
ctr=0

#resampled.iloc[index, column] = value;

# print(df.values)
from collections import Counter
sensorSamplesCount = Counter()

for record in df.values:
# record[0] - timestamp, record[1] - ID2, record[2] - RSSI, record[3] - dBm, record[4] - Label
    if (record[0] <= sampleStartTime):
        sensorSamplesCount[record[1]] += 1
        resampled.iloc[ctr, record[1]-1] = (resampled.iloc[ctr, record[1]-1] + record[2]) / sensorSamplesCount[record[1]] 
            #fill at index#ctr for sensor #record[1] value of record[2]
        resampled.iloc[ctr, -1] = record[4] #fill label
    else:
        ctr=ctr+1
        sensorSamplesCount = Counter() #reset sensor sample counts
        sampleStartTime = record[0] + timedelta(seconds=2)
        sensorSamplesCount[record[1]] += 1
        resampled.iloc[ctr, record[1]-1] = (resampled.iloc[ctr, record[1]-1] + record[2]) / sensorSamplesCount[record[1]] 
            #fill at index#ctr for sensor #record[1] value of record[2]
        resampled.iloc[ctr, -1] = record[4] #fill label


# print(sensorSamplesCount)
# print(sum(sensorSamplesCount.values()))
lastIndex = (resampled['Label'] == 0).argmax() #find first non-valid record and record index


writeResample = True
if (writeResample == True):
    resampled.to_csv('resampledData_160320.csv', index=False) 
    print("FILE WRITTEN!")
    
resampled.head(100)

CODE: 1T4Z
                   Time  ID2  RSSI     dBm Label
0  2020-03-16 17:19:27    7   177 -28.102     A
1  2020-03-16 17:19:27    6   177 -28.102     A
2  2020-03-16 17:19:27    2   181 -26.206     A
3  2020-03-16 17:19:27   10   180 -26.680     A
4  2020-03-16 17:19:28    5   177 -28.102     A
5  2020-03-16 17:19:28    3   188 -22.888     A
6  2020-03-16 17:19:28    1   186 -23.836     A
7  2020-03-16 17:19:28    4   177 -28.102     A
8  2020-03-16 17:19:29    7   177 -28.102     A
9  2020-03-16 17:19:29   10   177 -28.102     A
10 2020-03-16 17:19:29    2   193 -20.518     A
11 2020-03-16 17:19:30    3   187 -23.362     A
12 2020-03-16 17:19:30    1   185 -24.310     A
13 2020-03-16 17:19:30    5   177 -28.102     A
14 2020-03-16 17:19:30    4   177 -28.102     A
FILE WRITTEN!


,1,2,3,4,5,6,7,8,9,10,11,Label
0,186.0,187.0,188.0,177.0,177.0,177.0,177.0,0.0,0.0,178.5,0.0,A
1,185.0,199.0,185.5,177.0,177.0,0.0,177.0,177.0,0.0,187.0,181.0,A
2,178.0,191.5,179.0,177.0,180.0,181.0,0.0,178.0,0.0,184.0,181.0,A
3,186.0,185.0,179.5,177.0,178.0,181.0,177.0,177.0,0.0,180.0,179.5,A
4,187.0,189.0,181.0,177.0,178.0,181.0,177.0,179.5,0.0,0.0,179.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,181.0,180.0,180.0,178.0,184.0,177.0,180.0,177.0,0.0,188.0,180.5,A
96,183.5,190.0,180.0,178.0,0.0,181.0,180.0,0.0,0.0,177.0,0.0,A
97,187.0,190.0,180.0,177.0,0.0,181.0,0.0,0.0,0.0,177.0,177.0,A
98,187.0,190.0,179.0,177.0,178.0,0.0,177.0,181.0,0.0,191.0,181.0,A


In [3]:
resampled = pd.read_csv('JJResampled_160320.csv')
lastIndex = len(resampled)
dataset = resampled
ips_data = dataset.values
#dataset.iloc[rowrange, columnrange]

ips_data = dataset.iloc[0:lastIndex, :11]
ips_labels = dataset.iloc[:lastIndex, 11:]
X = ips_data
y = ips_labels
k_value = math.ceil(math.sqrt(len(ips_data)))
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), shuffle = True,test_size=0.3) 
# print("estimated k value: ", k_value)
# print("total train: ", len(X_train), 
#       '\n', X_train, '\n', y_train)
# print("total tests: ", len(X_test),
#       '\n', X_test, '\n', y_test)

print(type(y_train))
y_train = y_train.reshape(len(y_train),)
print(y_train)
# try:
#     from collections.abc import Iterable
# except ImportError:
#     from collections import Iterable

# def flatten(items):
#     for x in items:
#         if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
#             for sub_x in flatten(x):
#                 yield sub_x
#         else:
#             yield x

                     
# y_train = list(flatten(y_train))
# y_test = list(flatten(y_test))
# type(y_train)


<class 'numpy.ndarray'>
['D' 'D' 'A' ... 'C' 'B' 'B']


# Parameter Optimisation

In [4]:
# k_range = list(range(1, 31))
# print("k_range: ", k_range)

# classifier = KNeighborsClassifier()

# param_grid = dict(n_neighbors=k_range)
# print("param_grid: ", param_grid)

# grid = GridSearchCV(classifier, param_grid, cv=10, scoring='accuracy')

# grid.fit(X,y)
# print("best_estimator: ", grid.best_estimator_)

# KNN Algorithm

In [5]:
n_samples = 20

#Function calculates euclidean distance between two n-dimensional data instances 
def euclideanDistance(instance1, instance2):
    #handles if instances are lists or tuples:
    instance1 = np.array(instance1) 
    instance2 = np.array(instance2)
    
    '''
    https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html
    uses 2-norm frobenius norm and returns euclidean distance
    '''
    return np.linalg.norm(instance1 - instance2) #euclidean distance

In [6]:
#Function finds nearest neighbours; nearest -> smallest euclidean distance
def get_neighbors(training_set, 
                  labels, 
                  test_instance, 
                  k, 
                  distance=euclideanDistance):
    """
    get_neighbors calculates a list of the k nearest neighbors
    of an instance 'test_instance'.
    The list neighbors contains 3-tuples with  
    (index, dist, label)
    where
    index    is the index from the training_set, 
    dist     is the distance between the test_instance and the 
             instance training_set[index]
    distance is a reference to a function used to calculate the 
             distances
    """
    distances = [] #empty distance array
    
    #calculates euclidean distance between test_instance and ALL other instances in training_set
    for index in range(len(training_set)):
        dist = euclideanDistance(test_instance, training_set[index])
        distances.append((training_set[index], dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors # The list neighbors contains 3-tuples with (index, dist, label)

In [7]:
# TEST OUTPUT

outArray = []
for i in range(n_samples):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
#     print(i,
#           X_test[i],
#           y_test[i],
#           neighbors)
    
    outArray.append([i,
          X_test[i],
          y_test[i],
          neighbors])

out_df = pd.DataFrame(outArray, columns=['i', 'X_test', 'y_test', 'neighbours'])
out_df.head()
# out_df.tail()

,i,X_test,y_test,neighbours
0,0,"[186.0, 192.0, 193.0, 0.0, 0.0, 0.0, 0.0, 179....",[B],"[([184.0, 191.0, 194.0, 0.0, 0.0, 0.0, 0.0, 17..."
1,1,"[187.0, 193.0, 193.0, 177.0, 178.0, 185.0, 177...",[B],"[([187.0, 193.0, 193.0, 177.0, 179.0, 185.0, 1..."
2,2,"[0.0, 178.0, 182.0, 187.0, 186.0, 0.0, 0.0, 18...",[D],"[([0.0, 178.0, 181.0, 188.0, 185.0, 0.0, 0.0, ..."
3,3,"[184.0, 188.0, 177.0, 179.0, 179.0, 177.0, 0.0...",[A],"[([183.0, 189.0, 178.0, 177.0, 179.0, 177.0, 0..."
4,4,"[182.0, 179.0, 179.0, 179.0, 177.0, 177.0, 180...",[A],"[([181.0, 178.0, 178.0, 178.0, 184.0, 177.0, 1..."


In [8]:
from collections import Counter

#Function enables voting mechanic in KNN for Classification according to majority class vote
def vote(neighbors):
    class_counter = Counter() #A Counter is a dict subclass for counting hashable objects. 
    for neighbor in neighbors:
        class_counter[neighbor[2]] += 1 #neighbor[2] -> label for neighbor(s)
    return class_counter.most_common(1)[0][0]

In [9]:
for i in range(n_samples):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    print("index: ", i, 
          ", result of vote: ", vote(neighbors), 
          ", label: ", y_train[i], 
          ", data: ", X_train[i])

index:  0 , result of vote:  B , label:  D , data:  [  0. 177. 177. 185. 185.   0. 177.   0.   0. 181. 193.]
index:  1 , result of vote:  B , label:  D , data:  [  0.   0. 177. 183. 185. 188. 177.   0.   0. 177. 193.]
index:  2 , result of vote:  D , label:  A , data:  [  0. 189. 177. 177. 178.   0.   0. 182.   0. 191. 181.]
index:  3 , result of vote:  B , label:  D , data:  [  0. 177. 185. 181.   0. 185.   0. 177.   0. 184. 195.]
index:  4 , result of vote:  A , label:  A , data:  [180. 180. 177. 179. 184.   0. 181. 177.   0. 190. 180.]
index:  5 , result of vote:  A , label:  A , data:  [183. 190. 180.   0.   0.   0. 177. 181.   0. 177. 179.]
index:  6 , result of vote:  A , label:  A , data:  [187. 189. 181. 177.   0. 181. 177.   0.   0.   0. 177.]
index:  7 , result of vote:  D , label:  B , data:  [187. 190. 193. 177.   0. 185. 177. 179.   0. 177. 177.]
index:  8 , result of vote:  A , label:  D , data:  [177. 178. 182. 188. 185.   0.   0. 187.   0. 186. 197.]
index:  9 , result 

In [10]:
#Function returns vote 'probability' - i.e. distribution/percentage majority vote
def vote_prob(neighbors):
    class_counter = Counter() # Counter object - https://docs.python.org/2/library/collections.html
    for neighbor in neighbors:
        class_counter[neighbor[2]] += 1 #add to count of target (class)
        
    # aggregates into tuples ~ zip(*iterables), 
    # Return a list of the n most common elements and their counts from the most common to the least.    
    labels, votes = zip(*class_counter.most_common()) #returns list of sorted most common [labels], [votes]
    #print("L|V: ", labels, votes)
    #print("Class Counter: ", class_counter.most_common)
    winner = class_counter.most_common(1)[0][0]       #majority label
    votes4winner = class_counter.most_common(1)[0][1] #majority vote count
    return winner, votes4winner/sum(votes)            #returns majority label, majority proportion

In [11]:
resultsKNN = pd.DataFrame(0, index=range(len(X_test)), columns=['index', 'data', 'vote_result', 'vote_prob', 'label', 'prediction'])

indexArray = []
dataArray = []
voteArray = []
probArray = []
labelArray = []
predArray = []
for i in range(len(X_test)):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    indexArray.append(i)
    dataArray.append(X_test[i])
    voteArray.append(vote(neighbors))
    probArray.append(vote_prob(neighbors))
    labelArray.append(y_test[i][0])
    predArray.append("CORRECT" if (vote(neighbors) == y_test[i]) else "WRONG")
#     print("index: ", i, 
#           ", result of vote: ", vote(neighbors), 
#           ", vote_prob: ", vote_prob(neighbors), 
#           ", label: ", y_test[i], "prediction: "
#         "CORRECT" if (vote(neighbors) == y_test[i]) else "WRONG"
#           ", data: ", X_test[i])

resultsKNN['index'] = indexArray
resultsKNN['data'] = dataArray
resultsKNN['vote_result'] = voteArray
resultsKNN['vote_prob'] = probArray
resultsKNN['label'] = labelArray 
resultsKNN['prediction'] = predArray


resultsKNN.to_csv('JJ_KNN_160320.csv', index=False) 
resultsKNN.head(20)

,index,data,vote_result,vote_prob,label,prediction
0,0,"[186.0, 192.0, 193.0, 0.0, 0.0, 0.0, 0.0, 179....",B,"(B, 0.6304347826086957)",B,CORRECT
1,1,"[187.0, 193.0, 193.0, 177.0, 178.0, 185.0, 177...",B,"(B, 0.8260869565217391)",B,CORRECT
2,2,"[0.0, 178.0, 182.0, 187.0, 186.0, 0.0, 0.0, 18...",D,"(D, 0.782608695652174)",D,CORRECT
3,3,"[184.0, 188.0, 177.0, 179.0, 179.0, 177.0, 0.0...",B,"(B, 0.5652173913043478)",A,WRONG
4,4,"[182.0, 179.0, 179.0, 179.0, 177.0, 177.0, 180...",A,"(A, 0.7391304347826086)",A,CORRECT
5,5,"[184.0, 190.0, 178.0, 177.0, 0.0, 0.0, 177.0, ...",A,"(A, 0.5217391304347826)",A,CORRECT
6,6,"[177.0, 177.0, 177.0, 179.0, 0.0, 181.0, 179.0...",A,"(A, 0.6304347826086957)",C,WRONG
7,7,"[0.0, 0.0, 177.0, 185.0, 0.0, 183.0, 177.0, 0....",D,"(D, 0.5)",D,CORRECT
8,8,"[188.0, 192.0, 193.0, 177.0, 178.0, 0.0, 177.0...",A,"(A, 0.6521739130434783)",B,WRONG
9,9,"[186.0, 194.0, 199.0, 181.0, 0.0, 179.0, 0.0, ...",B,"(B, 0.5434782608695652)",B,CORRECT


# Weighted KNN

In [12]:
#Function for weighted KNN voting mechanic; harmonic weights based on ranking of datapoint (vote += 1/rank)
def vote_harmonic_weights(neighbors, all_results=True):
    class_counter = Counter()
    number_of_neighbors = len(neighbors)
    for index in range(number_of_neighbors):
        #weighted count of votes
#         print("Index: ", index)
#         print(class_counter[neighbors[index][2]])
#         print(neighbors[index][2][0])
        class_counter[neighbors[index][2][0]] += ( 1/(index+1) ) #add (1/ neighbour_rank) for each vote; index+1 = neighbour_rank
    labels, votes = zip(*class_counter.most_common())
    #print(labels, votes)
    winner = class_counter.most_common(1)[0][0]
    votes4winner = class_counter.most_common(1)[0][1]
    if all_results:
        total = sum(class_counter.values(), 0.0)
        
        for key in class_counter:
             class_counter[key] /= total #returns vote proportion for key ("class") in class_counter
        return winner, class_counter.most_common()
    else:
        return winner, votes4winner / sum(votes)

In [13]:
resultsRWKNN = pd.DataFrame(0, index=range(len(X_test)), columns=['index', 'data', 'label', 'vote_result', 'prediction'])

indexArray = []
dataArray = []
labelArray = []
voteArray = []
predArray = []

for i in range(len(X_test)):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    
    indexArray.append(i)
    dataArray.append(X_test[i])
    labelArray.append(y_test[i])
    voteArray.append(vote_harmonic_weights(neighbors,all_results=True)[0])
    predArray.append("CORRECT" if (vote_harmonic_weights(neighbors,all_results=True)[0] == y_test[i]) else "WRONG")
    
#     print("index:", i, 
#           ",result of vote: ", 
#           vote_harmonic_weights(neighbors,
#                                 all_results=True), 
#           ",label: ", y_test[i], 
#         ",prediction: ",
#         "CORRECT" if ( vote_harmonic_weights(neighbors,
#                                 all_results=True)[0] == y_test[i]) else "WRONG")
    
resultsRWKNN['index'] = indexArray
resultsRWKNN['data'] = dataArray
resultsRWKNN['label'] = labelArray
resultsRWKNN['vote_result'] = voteArray
resultsRWKNN['prediction'] = predArray


resultsRWKNN.to_csv('JJ_RankWKNN_160320.csv', index=False) 

resultsRWKNN


,index,data,label,vote_result,prediction
0,0,"[186.0, 192.0, 193.0, 0.0, 0.0, 0.0, 0.0, 179....",[B],B,CORRECT
1,1,"[187.0, 193.0, 193.0, 177.0, 178.0, 185.0, 177...",[B],B,CORRECT
2,2,"[0.0, 178.0, 182.0, 187.0, 186.0, 0.0, 0.0, 18...",[D],D,CORRECT
3,3,"[184.0, 188.0, 177.0, 179.0, 179.0, 177.0, 0.0...",[A],A,CORRECT
4,4,"[182.0, 179.0, 179.0, 179.0, 177.0, 177.0, 180...",[A],A,CORRECT
...,...,...,...,...,...
609,609,"[185.0, 0.0, 198.0, 180.0, 177.0, 178.0, 0.0, ...",[B],B,CORRECT
610,610,"[0.0, 177.0, 177.0, 185.0, 0.0, 183.0, 177.0, ...",[D],D,CORRECT
611,611,"[184.0, 190.0, 179.0, 177.0, 0.0, 177.0, 0.0, ...",[A],B,WRONG
612,612,"[184.0, 189.0, 0.0, 179.0, 0.0, 0.0, 0.0, 177....",[A],A,CORRECT


# Distance Weighted KNN

In [14]:
#Function for distance weighted KNN voting mechanic; weight based on euclidean distance (vote += 1/distance)
def vote_distance_weights(neighbors, all_results=True):
    debug = False
    class_counter = Counter()
    number_of_neighbors = len(neighbors)
    for index in range(number_of_neighbors):
        dist = neighbors[index][1]
        label = neighbors[index][2]
        if (debug == True):
            print("Neighbour record: Label is ", label, "with distance ", dist)
            print("Existing count for ", label, "is ", class_counter[label[0]])
            print("Adding count by ", (1 / (1 if dist == 0 else dist)))
        class_counter[label[0]] += (1 / (1 if dist == 0 else dist)) #sensitivity of distance weight can be adjusted here.
        
        
    labels, votes = zip(*class_counter.most_common())
    #print(labels, votes)
    winner = class_counter.most_common(1)[0][0]
    votes4winner = class_counter.most_common(1)[0][1]
    if (debug == True):
        print("COUNTER: ", class_counter)
        print("----------------------------------------------------")
        print("winner: ", winner)
        print("votes: ", votes4winner)
    
    if all_results:
        total = sum(class_counter.values(), 0.0)
        
        for key in class_counter:
            class_counter[key] /= total
        if (debug == True):
            print("total votes: ", total)
            print("AFTER NORMALISING: ", class_counter)
        return winner, class_counter.most_common()
    else:
        return winner, votes4winner / sum(votes)

In [20]:
for i in range(5):
    neighbors = get_neighbors(X_test, 
                              y_test, 
                              X_train[i], 
                              5, 
                              distance=euclideanDistance)
    res = vote_distance_weights(neighbors, all_results=True)
    
    print("index: ", i, 
      ", result of vote: ", res,
    ",label: ", y_test[i], 
    ",prediction: ",
    "CORRECT" if ( res[0] == y_test[i]) else "WRONG")
    print("\n\n")

index:  0 , result of vote:  ('D', [('D', 1.0)]) ,label:  ['B'] ,prediction:  WRONG



index:  1 , result of vote:  ('D', [('D', 1.0)]) ,label:  ['B'] ,prediction:  WRONG



index:  2 , result of vote:  ('D', [('D', 0.7750999196955234), ('B', 0.22490008030447659)]) ,label:  ['D'] ,prediction:  CORRECT



index:  3 , result of vote:  ('D', [('D', 1.0)]) ,label:  ['A'] ,prediction:  WRONG



index:  4 , result of vote:  ('A', [('A', 1.0)]) ,label:  ['A'] ,prediction:  CORRECT





In [19]:
resultsDWKNN = pd.DataFrame(0, index=range(len(X_test)), columns=['index', 'data', 'label', 'vote_result', 'prediction'])

writeFile = False
indexArray = []
dataArray = []
labelArray = []
voteArray = []
predArray = []

# need to normalise distance!!!!

for i in range(len(X_test)):
    neighbors = get_neighbors(X_test, 
                              y_test, 
                              X_train[i], 
                              k_value, 
                              distance=euclideanDistance)
    vote_distance_weights(neighbors, all_results=True)
    
    indexArray.append(i)
    dataArray.append(X_test[i])
    labelArray.append(y_test[i])
    voteArray.append(vote_distance_weights(neighbors,all_results=True)[0])
    predArray.append("CORRECT" if (vote_distance_weights(neighbors,all_results=True)[0] == y_test[i]) else "WRONG")

#     print("index: ", i, 
#           ", result of vote: ", vote_distance_weights(neighbors,
#                                                       all_results=True),
#         ",label: ", y_test[i], 
#         ",prediction: ",
#         "CORRECT" if ( vote_harmonic_weights(neighbors,
#                                 all_results=True)[0] == y_test[i]) else "WRONG")
    
resultsDWKNN['index'] = indexArray
resultsDWKNN['data'] = dataArray
resultsDWKNN['label'] = labelArray
resultsDWKNN['vote_result'] = voteArray
resultsDWKNN['prediction'] = predArray


if (writeFile == True):
    resultsDWKNN.to_csv('JJ_DistanceWKNN_160320.csv', index=False) 
    print("FILE WRITTEN!")
resultsDWKNN



,index,data,label,vote_result,prediction
0,0,"[186.0, 192.0, 193.0, 0.0, 0.0, 0.0, 0.0, 179....",[B],D,WRONG
1,1,"[187.0, 193.0, 193.0, 177.0, 178.0, 185.0, 177...",[B],D,WRONG
2,2,"[0.0, 178.0, 182.0, 187.0, 186.0, 0.0, 0.0, 18...",[D],D,CORRECT
3,3,"[184.0, 188.0, 177.0, 179.0, 179.0, 177.0, 0.0...",[A],D,WRONG
4,4,"[182.0, 179.0, 179.0, 179.0, 177.0, 177.0, 180...",[A],A,CORRECT
...,...,...,...,...,...
609,609,"[185.0, 0.0, 198.0, 180.0, 177.0, 178.0, 0.0, ...",[B],C,WRONG
610,610,"[0.0, 177.0, 177.0, 185.0, 0.0, 183.0, 177.0, ...",[D],B,WRONG
611,611,"[184.0, 190.0, 179.0, 177.0, 0.0, 177.0, 0.0, ...",[A],A,CORRECT
612,612,"[184.0, 189.0, 0.0, 179.0, 0.0, 0.0, 0.0, 177....",[A],B,WRONG


In [17]:
from pandas import read_csv, pivot_table

def get_seconds(time):
    hours, mins, secs = time.split(":")
    return int(hours) * 3600 + int(mins) * 60 + int(secs)


readings = read_csv("BLEData_130320.csv")[["Time", "ID2", "RSSI", "Label"]]
readings.Time = readings.Time.transform(get_seconds)
readings.Time = (readings.Time - readings.Time.min()) // 2
readings.Time -= (readings.Time.diff().fillna(0) - 1).clip(lower=0).cumsum()
readings.ID2 = readings.ID2.transform(lambda hexadecimal: int(hexadecimal, 16))
resampled_readings = readings.groupby(["Time", "Label", "ID2"]).mean().reset_index()
pivoted_readings = pivot_table(resampled_readings, values="RSSI", columns=["ID2"], index=["Time", "Label"]).fillna(0)
print(pivoted_readings)

ID2              1      2      3      4      5      6      7      8    9   \
Time   Label                                                                
0.0    A      186.0  181.0  188.0  177.0  177.0  177.0  177.0    0.0  0.0   
1.0    A      185.0  193.0  187.0  177.0  177.0    0.0  177.0    0.0  0.0   
2.0    A        0.0  199.0  184.0  177.0    0.0    0.0  177.0  177.0  0.0   
3.0    A      178.0  192.0  179.0    0.0  180.0  181.0    0.0  178.0  0.0   
4.0    A      185.0  191.0  181.0  177.0  178.0    0.0    0.0  178.0  0.0   
...             ...    ...    ...    ...    ...    ...    ...    ...  ...   
2039.0 D        0.0  177.0  177.0  183.0    0.0    0.0  177.0    0.0  0.0   
2040.0 D        0.0  177.0    0.0  183.0  184.0  186.0  178.0    0.0  0.0   
2041.0 D        0.0    0.0    0.0  182.0  189.0  185.0    0.0  178.0  0.0   
2042.0 D        0.0  179.0  177.0  181.0    0.0    0.0  177.0  179.0  0.0   
2043.0 D        0.0  181.0    0.0  182.0  188.0  177.0    0.0  180.0  0.0   